***Resources:***
- https://dototot.com/how-to-write-a-twitter-bot-with-python-and-tweepy/
- https://dototot.com/reply-tweets-python-tweepy-twitter-bot/
- https://dev.twitter.com



In [ ]:
!pip install tweepy
!pip install requests
import tweepy, time, sys, requests, json

***Get Twitter App Info***
- Authorization 
- Connect with the API

API Key
- m3u7TiHyKFF7qTMp4U6lhRsGW

API Secret Key
- XFrIi7fjm7yyHIcrsqxo5DcLCrq75VqY4F6MtEDVnfG1ZWLhDn

Bearer Token
- AAAAAAAAAAAAAAAAAAAAAAZTKAEAAAAAVXFcTkLWo%2F5VOSLtkr0PCKlroAs%3DgmFZXlQntyMOy8LPxsQ3tLxH1I4fPC1hFI0FnAQiqoKazVj82W

In [ ]:
#TODO: visit https://dev.twitter.com to create an application and get your keys
keys = dict(
	consumer_key =          'm3u7TiHyKFF7qTMp4U6lhRsGW',
	consumer_secret =       'XFrIi7fjm7yyHIcrsqxo5DcLCrq75VqY4F6MtEDVnfG1ZWLhDn',
	access_token =          '1328837627898900482-oszm8Rr1jfEjRrVoHvqtGQMGc2hOFs',
	access_token_secret =   'z6gfXwbwctGrHfOipC7NkGIOZcAPOl0HfxdlXEf2kWwSv',
)




In [ ]:
CONSUMER_KEY = keys['consumer_key']
CONSUMER_SECRET = keys['consumer_secret']
ACCESS_TOKEN = keys['access_token']
ACCESS_TOKEN_SECRET = keys['access_token_secret']

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
def create_session():
  r = requests.post('https://api.us-south.assistant.watson.cloud.ibm.com/instances/abadaf22-9195-425a-aa9a-3e86f1cb4a28/v2/assistants/cca390d2-78f4-442c-b2e4-6275fe74ed82/sessions?version=2020-04-01', auth=('apikey', '2Vaj7UT5DHz6Kzco18u23jrEZEpry_79Xk7Gz3TF1bpA'))
  return r.json()['session_id']

In [ ]:
def WatsonAnalyze(sn, ogmsg):
  #Set up Watson account
  session_id = create_session()
  entities = {'contactWithCovid': 0, 'CoronavirusSymptom': 0, 'mentalHealthSymptom': 0, 'nonCovidSymptom': 0, 'healthEmergancy': 0}

  #Try for health emergancy
  ogresp = send_message(ogmsg, session_id)
  if ogresp['intents'] == "healthEmergancy":
    #Health Emergency
    return "Hello {},".format(sn) + ogresp
  #Else do nothing and move on

  #Iterate through until no words left
  words = ogmsg.split(" ")
  while len(words) > 0:
    count = 1
    unhappy = True
    while unhappy:
      chunk = iterateOverWords(words, count)
      response = send_message(chunk, session_id)['entities']
      for entity in response:
        entities[entity['entity']] += 1

      if len(response) > 0 or count > 9 or count + 1 > len(words):
        unhappy = False #move on 
        words = words[count:]
        
      else:
      #Else we don't move on and increase count
        count = count + 1
    
  #Send "EOS" NEVERMIND. Don't have to do that now that counters are in python
  #msgDict = send_message("EOS", session_id)
  msg = ""
  #Severe Warning
  if(entities['CoronavirusSymptom'] > 3 or entities['contactWithCovid'] > 1):
    msg = "It is extremely likely that you have contracted COVID-19. Call your doctor immediately, distance from friends and family, and do not engage in contact with others for any reason."
  #Advise Coution
  elif(entities['CoronavirusSymptom'] > 1 or entities['contactWithCovid'] == 1):
    msg = "It seems like you may have contracted COVID-19. Though we can't say for sure, it is very important to be cautious and to distance from friends and family. Please get a test as soon as possible."
  #Potential
  elif(entities['CoronavirusSymptom'] == 1):
    msg = "We see some signs in your tweet that you may have COVID-19. Even with minor symptoms, it is important to wear a mask and limit your contact with others, and get a test as soon as possible."
  #Mental Health
  elif(entities['mentalHealthSymptom'] > 0):
    msg = "Though we don't see any evidence of COVID-19 in this tweet, you've listed signs of mental health issues. Please set up an online appointment with a mental health specialist. If you experience a mental health emergency, call 1-800-273-8255."
  #NonCovidSymptoms are prevelant
  elif(entities['nonCovidSymptom'] > 3):
    msg = "Looks like you're having a lot of medical symptoms that are unrelated to COVID-19! Please call a doctor immediatly to discuss your symptoms."
  #A few NonCovidSymptoms
  elif(entities['nonCovidSymptom'] > 0):
    msg = "Looks like you mentioned a few medical symptoms, however these are not signs of COVID-19. We reccomend speaking to a doctor, especially if your symptoms worsen."
  #No Evidence
  elif(entities['CoronavirusSymptom'] == 0 and entities['contactWithCovid'] == 0 and entities['mentalHealthSymptom'] == 0):
    msg = "We don't see any evidence that you have contracted COVID-19 from your tweet! Even so, it is very common to be an asymptomatic carrier, so it is still important to wear a mask and social distance."
  #Invalid
  else:
    msg = "We had some trouble reading your message, try rephrasing so that we can get you more conclusive results!"

  return "Hello @{},".format(sn) + msg

In [ ]:
def send_message(text, session_id):
  print('message to watson = {}'.format(text))
  data = json.dumps({"input": {"text": "{}".format(text)}})
  r = requests.post('https://api.us-south.assistant.watson.cloud.ibm.com/instances/abadaf22-9195-425a-aa9a-3e86f1cb4a28/v2/assistants/cca390d2-78f4-442c-b2e4-6275fe74ed82/sessions/{}/message?version=2020-04-01'.format(session_id), data=data, headers={'Content-Type': 'application/json'}, auth=('apikey', '2Vaj7UT5DHz6Kzco18u23jrEZEpry_79Xk7Gz3TF1bpA'))
  
  print('watson response json = {}'.format(r.json()))

  return r.json()['output']

In [ ]:
 #Iterate over split words
 #Words is the words 
 #index is the 1-10 number of words we're trying in a chunk. after 10 we give up and delete the word
 #Returns a string separated by whitespace of the first (index) words in the words array
def iterateOverWords(words, index):
  arr = []
  for i in range(0,index):
    arr.append(words[i])
  return " ".join(arr)


In [ ]:
# get a list of the tweets that @ our bot's handle
twts = api.search(q="@diagnosu")

for status in twts:
  print('status = {}'.format(status))
  # get screen name of person that @DiagnOSU
  sn = status.user.screen_name
  if sn != 'DiagnOSU':
    ogmsg = status.text
    print('screen name = {}'.format(sn))
    print('message = {}'.format(ogmsg))
    watson_response = WatsonAnalyze(sn, ogmsg)
    print('watson resp = {}'.format(watson_response))
    #m = WatsonAnalyze(sn, ogmsg)
    status = api.update_status(watson_response, in_reply_to_status_id=status.id)

status = Status(_api=<tweepy.api.API object at 0x7f840ad0dac8>, _json={'created_at': 'Thu Dec 03 22:52:37 +0000 2020', 'id': 1334631656645419009, 'id_str': '1334631656645419009', 'text': "@DiagnOSU I'm so hungry", 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'DiagnOSU', 'name': 'Diagnosu Team', 'id': 1328837627898900482, 'id_str': '1328837627898900482', 'indices': [0, 9]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': 1328837627898900482, 'in_reply_to_user_id_str': '1328837627898900482', 'in_reply_to_screen_name': 'DiagnOSU', 'user': {'id': 448629667, 'id_str': '448629667', 'name': 'Anna', 'screen_name': 'annaeamrhein', 'location': 'The Ohio State University', 'description': '', 'url': None, 'entities': {'description':

TweepError: ignored